<p><a href="https://colab.research.google.com/github/MIsaiko/SORCT_Pyomo/blob/main/Softmax_Pyomo.ipynb"><img align="left" src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab" title="Open in Google Colaboratory"></a>

## Imports

In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
import scipy.stats as stats
import scipy.optimize as optimize

import shutil
import sys
import os.path

if not shutil.which("pyomo"):
    !pip install -q pyomo
    assert(shutil.which("pyomo"))

if not (shutil.which("ipopt") or os.path.isfile("ipopt")):
    if "google.colab" in sys.modules:
        !wget -N -q "https://ampl.com/dl/open/ipopt/ipopt-linux64.zip"
        !unzip -o -q ipopt-linux64
    else:
        try:
            !conda install -c conda-forge ipopt 
        except:
            pass

assert(shutil.which("ipopt") or os.path.isfile("ipopt"))

from pyomo.environ import *
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

## Description

This notebook uses optimization tools [Pyomo](http://www.pyomo.org/) to implement the logistic regression and softmax regression then tests it by using [The breast cancer dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_breast_cancer.html) and [The Iris Dataset](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html).

## Logistic regression

In [2]:
def logistic_regression(X, y, w_reg = 0.001):
    n_instances, n_features = X.shape
    
    m = ConcreteModel()
    
    m.F = Set(initialize=list(range(n_features)))
    m.I = Set(initialize=list(range(n_instances)))
    
    train_set = {(i, f): X[i,f] for i in m.I for f in m.F}
    m.X = Param(m.I, m.F, within=Reals, initialize=train_set)
    m.y = Param(m.I, within=Reals, initialize={i: y[i] for i in m.I})
    
    m.beta = Var(m.F, within=Reals)

    def p_exp(model, i):
      return 1.0 / (1.0 + exp(-1*sum(model.beta[f]*model.X[i, f] for f in model.F)))
    m.p = Expression(m.I, rule=p_exp)
    
    # Objective:
    def cost_rule(model):
        cost = 0.0
        for i in model.I:
            cost += model.y[i]*log(model.p[i]) + (1.0-model.y[i])*log(1.0-model.p[i])
        reg = w_reg*sum(model.beta[f]**2 for f in model.F) # Regularization
        return -1*cost + reg
    m.cost = Objective(rule=cost_rule, sense=minimize)

    return m

def lr_predict(model, X):
  pred = []
  for i in range(X.shape[0]):
    pred.append(round(1.0 / (1.0 + np.exp(-1*sum(model.beta[f]()*X[i, f] for f in model.F)))))
  return pred

## Softmax regression

In [3]:
def softmax_regression(X, y, k, w_reg = 0.001):
    n_instances, n_features = X.shape
    
    m = ConcreteModel()
    
    m.F = Set(initialize=list(range(n_features)))
    m.I = Set(initialize=list(range(n_instances)))

    m.k = Set(initialize=list(range(k)))
    
    train_set = {(i, f): X[i,f] for i in m.I for f in m.F}
    m.X = Param(m.I, m.F, within=Reals, initialize=train_set)
    m.y = Param(m.I, within=Reals, initialize={i: y[i] for i in m.I})
    
    m.beta = Var(m.k, m.F, within=Reals)

    def p_exp(model, i, ki):
        beta_list = []
        for k_i in model.k:
            beta_i = exp(sum(model.beta[k_i, f]*model.X[i, f] for f in model.F))
            beta_list.append(beta_i)
        return [b/sum(beta_list) for b in beta_list][ki]
    m.p = Expression(m.I, m.k, rule=p_exp)
    
    # Objective:
    def cost_rule(model):
        cost = 0.0
        for i in model.I:
            for ki in model.k:
                if model.y[i] == ki:
                    cost += log(model.p[i,ki])
        reg = w_reg*sum(model.beta[k_i,f]**2 for k_i in model.k for f in model.F)
        return -1*cost + reg
    m.cost = Objective(rule=cost_rule, sense=minimize)

    return m

def softmax_predict(model, X):
    pred = []
    for i in range(X.shape[0]):
        beta_list = []
        for k_i in model.k:
            beta_i = np.exp(sum(model.beta[k_i, f]()*X[i, f] for f in model.F))
            beta_list.append(beta_i)
        pred_all = [b/sum(beta_list) for b in beta_list]
        pred.append(pred_all.index(max(pred_all)))

    return pred

# Real Dataset

## LR

In [4]:
# load a binary classification dataset

X, y = datasets.load_breast_cancer(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [5]:
# train logistic regression model using ipopt with regularization weight of 0.5

lr_model = logistic_regression(X_train, y_train, w_reg=0.5)
solver = SolverFactory('ipopt')
# solver.options['halt_on_ampl_error_solver'] = 'yes'
results = solver.solve(lr_model, tee=True)
print(results)

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      465

Total number of variables............................:       30
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Tot

In [6]:
print("Train accuracy: ", accuracy_score(y_train, lr_predict(lr_model, X_train)))
print("Test accuracy: ", accuracy_score(y_test, lr_predict(lr_model, X_test)))

Train accuracy:  0.960093896713615
Test accuracy:  0.958041958041958


## Softmax

In [7]:
# load Iris dataset
X, y = datasets.load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

In [8]:
softmax_model = softmax_regression(X_train, y_train, 3, w_reg=0.5)
solver = SolverFactory('ipopt')
results = solver.solve(softmax_model, tee=True)
print(results)

Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:       78

Total number of variables............................:       12
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        0
                     variables with only upper bounds:        0
Tot

In [9]:
print("Train accuracy: ", accuracy_score(y_train, softmax_predict(softmax_model, X_train)))
print("Test accuracy: ", accuracy_score(y_test, softmax_predict(softmax_model, X_test)))

Train accuracy:  0.9821428571428571
Test accuracy:  0.9210526315789473
